In [3]:
######### poznan street read image ###########
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
%matplotlib inline

def Evaluate(pre_disp_path, gt_depth_path,name):
    min_depth = 0
    max_depth = 178
    
    gt_path = gt_depth_path
    
    rms     = np.zeros(num_samples, np.float32)
    log_rms = np.zeros(num_samples, np.float32)
    abs_rel = np.zeros(num_samples, np.float32)
    sq_rel  = np.zeros(num_samples, np.float32)
    d1_all  = np.zeros(num_samples, np.float32)
    a1      = np.zeros(num_samples, np.float32)
    a2      = np.zeros(num_samples, np.float32)
    a3      = np.zeros(num_samples, np.float32)

    def compute_errors(gtt, predd):
        gt = gtt/256
#         print("gt.max",gt.max())
#         print("gt.min",gt.min())
        pred = predd/256
#         print("pred.max",pred.max())
#         print("pred.min",pred.min())
        thresh = np.maximum((gt / pred), (pred / gt))
        a1 = (thresh < 1.25   ).mean()
        a2 = (thresh < 1.25 ** 2).mean()
        a3 = (thresh < 1.25 ** 3).mean()

        rmse = (gt - pred) ** 2
        rmse = np.sqrt(rmse.mean())
        rmse_log = (np.log(gt) - np.log(pred)) ** 2
        rmse_log = np.sqrt(rmse_log.mean())
        #abs_rel = np.mean(np.abs(gt - pred) / gt)
        abs_rel = np.mean(np.abs(gt - pred))
        sq_rel = np.mean(((gt - pred)**2) / gt)

        return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3


    def load_gt_depth_poznan(path, num_samples):
        gt_depths = []
        for i in range (num_samples):
            depth = cv2.imread(path +"Poznan_Street_4_"+str(i+1).zfill(3) + "_depth.png", 0)
            depth1 = cv2.resize(depth, (512, 256))
            depth = depth1.astype(np.float32)
            #depth = normalization(depth)
            gt_depths.append(depth)
        return gt_depths    
    
 
    
    def normalization(images):
        normal_img = ((images-images.min())/(images.max()-images.min())*255)+1
        return normal_img
    
    
    gt_depths = load_gt_depth_poznan(gt_path, num_samples)
    pred_depths = np.load(pre_disp_path)
    
    for i in range(num_samples):
        gt_depth = gt_depths[i]
        #pred_depth = pred_depths[i]
        pred_depth = pred_depths[i]*2701.35
        #print(pred_depth.max())
        pred_depth[pred_depth < min_depth] = min_depth
        pred_depth[pred_depth > max_depth] = max_depth
        a=np.abs(gt_depth-pred_depth)
#         cv2.imwrite('%s.png'%i,a)
#         cv2.imwrite('%smy.png'%i,pred_depth-1)
        #print("gt_depth",gt_depth)
        #print("pred_depth",pred_depth)
        mask = gt_depth > 0
        abs_rel[i], sq_rel[i], rms[i], log_rms[i], a1[i], a2[i], a3[i] = compute_errors(gt_depth[mask], 
                                                                                        pred_depth[mask])

        #abs_rel[i], sq_rel[i], rms[i], log_rms[i], a1[i], a2[i], a3[i] = compute_errors(gt_depth,pred_depth)
    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format
          ('abs_', 'sq_rel', 'rms', 'log_rms', 'a1', 'a2', 'a3'))
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.5f}, {:10.5f}, {:10.5f}".format
          (abs_rel.mean(), sq_rel.mean(), rms.mean(), log_rms.mean(), a1.mean(), a2.mean(), a3.mean()))
    
    outfile="/home/nung/Desktop/evaluate/evaluate_new.txt"
    file=open(outfile, "a")
    file.write('Poznan fg abs'+' '+name+'\n'+str(abs_rel.mean())+' '+str(sq_rel.mean())+' '+str(rms.mean())+' '+str(log_rms.mean())+' '+
               str(a1.mean())+' '+str(a2.mean())+' '+str(a3.mean())+'\n')

num_samples = 250
gt_path = '/home/nung/data/Poznan_Street/depth_Poznan_Street_4/'    


def Go_origin():
    name='origin'
    print('origin/'+str(num_samples)+' testing')
    pd_path = '/home/nung/aaaMONO/original/test/Poznan/disparities_pp.npy'
    Evaluate(pd_path,gt_path, name)

def Go_origin_resnet():
    name='origin_resnet'
    print('origin_resnet/'+str(num_samples)+' testing')
    pd_path = '/home/nung/aaaMONO/original/test_resnet/Poznan/disparities_pp.npy'
    Evaluate(pd_path,gt_path, name)    
    
def Go_STF_22():
    name='STF_22'
    print('STF_22fgabs/'+str(num_samples)+' testing')
    pd_path = '/home/nung/aaaMONO/forgate_gate/STFB-22/cPoznan/disparities_pp.npy'
    Evaluate(pd_path,gt_path, name)    

def Go_STF_55():
    name='STF_55'
    print('STF_55fgabs/'+str(num_samples)+' testing')
    pd_path = '/home/nung/aaaMONO/forgate_gate/STFB-55/cPoznan/disparities_pp.npy'
    Evaluate(pd_path,gt_path, name)    
    
def Go_STF_35():
    name='STF_35'
    print('STF_35fgabs/'+str(num_samples)+' testing')
    pd_path = '/home/nung/aaaMONO/forgate_gate/STFB-35/cPoznan/disparities_pp.npy'
    Evaluate(pd_path,gt_path, name)

def Go_BTF():
    name='BTF'
    print('BTFfgabs/'+str(num_samples)+' testing')
    pd_path = '/home/nung/aaaMONO/forgate_gate/BTFB/cPoznan/disparities_pp.npy'
    Evaluate(pd_path,gt_path, name)    
    
def Go_origin_ckvgg():
    name='origin_ckvgg'
    print('origin_ckvgg/'+str(num_samples)+' testing')
    pd_path = '/home/nung/aaaMONO/original/test_ck/Poznan/disparities_pp.npy'
    Evaluate(pd_path,gt_path, name)  
    
# Go_origin()
# Go_origin_ckvgg()
# Go_origin_resnet()
# Go_STF_22()
# Go_STF_55()
# Go_STF_35()
Go_BTF()



BTFfgabs/250 testing
      abs_,     sq_rel,        rms,    log_rms,         a1,         a2,         a3
    0.0374,     0.0125,     0.0500,     0.2909,    0.68936,    0.90985,    0.98001
